In [3]:
%run TransformCoordinates.ipynb

class QualityControl(TransformCoordinates):
    
    def __init__(self, tray, position, points, trayPosition, heights):
        TransformCoordinates.__init__(self, tray, position, points, heights)
        self.trayPosition = trayPosition
        self.degrees = [89.5, 89.0, 88.0, 0.]
        self.classify = {'0': 'GREEN', '1': 'YELLOW', '2': 'ORANGE', '3': 'RED'}
        self.centers = [0.025, 0.050, 0.100, 10.0]
        self.heights = [0.025, 0.050, 0.100, 10.0]
        self.nominalHeights = {'CuW': 3.100, 'PCB': 3.100}
        self.QC = []
        self.finalRating = ''
        self.notes = ''
        
    #Checking the quality of the angle of the placed component with respect to the tray position
    def angleQuality(self):
        self.degQC = -1
        for i, d in enumerate(self.degrees):
            if abs(self.deg[0]) > d or abs(self.deg[1]) > d:
                self.notes += 'The angle of the sensor is ' + self.classify[str(i)] + ' in position ' + self.trayPosition + '. '
                self.degQC = i
        self.QC.append(self.degQC)
        
    #Position quality checking for the component center
    def positionQuality(self):
        self.posQC = -1
        for i, p in enumerate(self.centers):
            if self.centerPosition < p:
                self.notes += 'The placement of the sensor is ' + self.classify[str(i)] + ' in position ' + self.trayPosition + '. '
                self.posQC = i
        self.QC.append(self.posQC)
              
    #Height measurement means/max/min is within the allowed nominal value for each module type
    def heightQuality(self):
        self.hgtQC = -1
        for i, h in enumerate(self.heights):
            if abs(self.hgt[0] - self.nominalHeights['CuW']) <= h:
                self.notes += 'The mean heights of the module are ' + self.classify[str(i)] + ' in position ' + self.trayPosition + '. '
                self.hgtQC = max(self.hgtQC, i)
            if self.hgt[1] < h:
                self.notes += 'The minimum height of the module is ' + self.classify[str(i)] + ' in position ' + self.trayPosition + '. '
                self.hgtQC = max(self.hgtQC, i)
            if self.hgt[0] < h:
                self.notes += 'The maximum height of the module is ' + self.classify[str(i)] + ' in position ' + self.trayPosition + '. '
                self.hgtQC = max(self.hgtQC, i)
        self.QC.append(self.hgtQC)
                
    #Final rating function that looks at each measurement type and classifies as GREEN/YELLOW/ORANGE/RED
    #We get a final rating from this also
    def Rating(self):
        if len(self.hgt): self.heightQuality()
        if len(self.deg): self.angleQuality()
        if self.centerPosition >= 0.: self.positionQuality()
        rate = max(self.QC)
        self.finalRating = self.classify[str(rate)]
        
        self.notes += '\nThe FINAL rating of the module in position ' + self.trayPosition + ' is ' + self.finalRating + '.'
        
        if rate >= 3:
            print('The components in position ' + self.trayPosition + ' has failed the QC.')
            print(self.notes + '\n')